<a href="https://colab.research.google.com/github/Vantage-AI/prorail_railway_fasteners/blob/master/ProRail_object_detection_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Object detection with the tensorflow object detection API
In this notebook we will train an object detection algorithm using the tensorflow object detection API.
Note that most of the object detection API still does not function with `tensorflow==2.*` or eager execution.

This notebook is all about getting it to work. So we ask you to fill in the gaps.

In [ ]:
%%capture
!pip install tf-slim
!pip install tensorflow-gpu==1.15.2

In [ ]:
from io import BytesIO
import glob
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
from PIL import Image
import pylab
import tensorflow
import tensorflow as tf
import time
import urllib

print(tf.__version__)
# tf.enable_eager_execution()

## Step 1: Install API
Installing the tensorflow object detection API.

In [ ]:
%%capture
# Install protoc
!wget -O protobuf.zip https://github.com/google/protobuf/releases/download/v3.0.0/protoc-3.0.0-linux-x86_64.zip -q
!unzip -o protobuf.zip
!rm protobuf.zip

In [ ]:
%%capture

#Clone TensorFlow Object Detection API v1.12.0
%mkdir /content/tensorflow
%cd /content/tensorflow
!rm -fr models
!git clone --depth 1 https://github.com/tensorflow/models.git
!rm -fr models/.git
    
# compile ProtoBuffers
%cd models/research
!/content/bin/protoc object_detection/protos/*.proto --python_out=.

In [ ]:
# Set environment variables
os.environ['AUTOGRAPH_VERBOSITY'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['PYTHONPATH']=f"{os.environ['PYTHONPATH']}:/content/tensorflow/models/research:/content/tensorflow/models/research/slim"
sys.path.append("/content/tensorflow/models/research")
sys.path.append("/content/tensorflow/models/research/slim")

In [ ]:
%cd /content
!python /content/tensorflow/models/research/object_detection/builders/model_builder_test.py

## 2. Prepare data
More information about how to create the tfrecord files can be found [here](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/using_your_own_dataset.md). It can be a a bit tedious, so we provide a script called `generate_tfrecord.py`.

The data gets split into separate sets for training, validation and testing. The model learns from the training data, but we use separate validation and test sets to make sure our model generalizes to unseen data!

In [ ]:
!wget https://github.com/Vantage-AI/prorail_railway_fasteners/raw/master/dataset.zip
!unzip -o dataset.zip -d /content

In [ ]:
# Create tf record files with a custom script called "generate_tfrecord"
for split in ['train', 'val', 'test']:
  os.environ['SPLIT'] = split
  !python /content/dataset/generate_tfrecord.py\
    --csv_input /content/dataset/${SPLIT}_annotations.csv\
    --output_path /content/dataset/${SPLIT}.record\
    --path_to_labels /content/dataset/labelmap.pbtxt\
    --img_path /content/dataset/${SPLIT}


## 3. Building a model
We will use a COCO pretrained model first, i.e. *Transfer Learning*. 
* [Here](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/configuring_jobs.md) is how to edit the config file.

In [ ]:
%%capture
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

model_name = "ssd_mobilenet_v2_coco_2018_03_29"
os.environ['MODEL'] = model_name
!mkdir models
!wget -O ${MODEL}.tar.gz  http://download.tensorflow.org/models/object_detection/${MODEL}.tar.gz -q
!tar -C /content/models -xvzf ${MODEL}.tar.gz
!rm ${MODEL}.tar.gz

# download category index
!wget -O models/mscoco_label_map.pbtxt https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/data/mscoco_label_map.pbtxt
category_index_coco = label_map_util.create_category_index_from_labelmap("models/mscoco_label_map.pbtxt", use_display_name=True)

In [ ]:
url = "https://i.pinimg.com/474x/4c/e8/04/4ce8048a99b43441dd58e91b4c3eadcb--choo-tractors.jpg"
data = BytesIO(urllib.request.urlopen(url).read())
image = pylab.imread(data, format='jpg').astype(np.uint8)
Image.fromarray(image)

Next, we will make a prediction. Lets see if the pretrained model can detect a sheep...?<sup>1</sup>.

<sub><sup><sup>1</sup>Since this we are running this in Graph mode (because we do everything in one notebook, you could run these cells in eager mode) it will be a bit tedious...</sup></sub>

In [ ]:
# Make a prediction
g = tf.Graph()
with g.as_default():
  new_model = tf.keras.models.load_model(os.path.join("/content/models", 
                                                      model_name, 
                                                      "saved_model"))
  new_model = new_model.signatures['serving_default']
  output_dict = new_model(tf.convert_to_tensor(np.expand_dims(image, 0)))  # here we make the prediction
  init_op = tf.group([tf.global_variables_initializer(), tf.tables_initializer()])
g.finalize()

# Create session and initialize.
session = tf.Session(graph=g)
session.run(init_op)
output_dict = session.run(output_dict)  # here we execute the graph

Next, we visualize the bounding box. Note that this tool will actually draw the box onto the array... so if you want to save it, make a copy.

In [ ]:
vis_util.visualize_boxes_and_labels_on_image_array(
  image,
  np.array(output_dict['detection_boxes'])[0],
  np.array(output_dict['detection_classes']).astype('int')[0],
  np.array(output_dict['detection_scores'])[0],
  category_index_coco,
  use_normalized_coordinates=True,
  line_thickness=3)

display(Image.fromarray(image))

## 4. Train your own model
Using pre-trained models is fun, but maybe you want to build your own model. Lets see how we can do just that.

In [ ]:
# !rm -r models/

In [ ]:
steps=30
os.environ["PIPELINE_CONFIG_PATH"]="/content/dataset/pipeline.config"
os.environ["MODEL_DIR"]=f"/content/models/my_model"
os.environ["NUM_TRAIN_STEPS"]=f"{steps}"  # 1000 steps takes about 10 minutes on a GPU and gives OK results for lightest mobilenet (ssd_mobilenet_v2)
os.environ["SAMPLE_1_OF_N_EVAL_EXAMPLES"]="1"

%load_ext tensorboard
%tensorboard --logdir /content/models/my_model

In [ ]:
# model_dir: where the new model will be saved
# Make sure this is different from your starting point!
%%capture
t1 = time.time()
!python tensorflow/models/research/object_detection/model_main.py \
    --pipeline_config_path=${PIPELINE_CONFIG_PATH} \
    --model_dir=${MODEL_DIR} \
    --num_train_steps=${NUM_TRAIN_STEPS} \
    --sample_1_of_n_eval_examples=${SAMPLE_1_OF_N_EVAL_EXAMPLES} \
    --alsologtostderr
t2 = time.time()

In [ ]:
# Takes about 
print(f"{t2 - t1:.1f}s")

## 5. Export your model for inference
This model has to be exported for inference, see [this link](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/exporting_models.md)

In [ ]:
export_dir=f"/content/models/my_model_inference"
os.environ['INPUT_TYPE']="image_tensor"
os.environ["TRAINED_CKPT_PREFIX"]=os.path.join(os.environ["MODEL_DIR"], f"model.ckpt-{steps}")  # extract with
os.environ['EXPORT_DIR']=export_dir

In [ ]:
%%capture
!python tensorflow/models/research/object_detection/export_inference_graph.py \
    --input_type=${INPUT_TYPE} \
    --pipeline_config_path=${PIPELINE_CONFIG_PATH} \
    --trained_checkpoint_prefix=${TRAINED_CKPT_PREFIX} \
    --output_directory=${EXPORT_DIR}

## 6. Inference
Now the fun part... Will it work??? Next, make some predictions on the the test set.

In [ ]:
if not os.path.exists("/content/models"):
  os.makedirs("/content/models")

In [ ]:
%matplotlib inline
image = pylab.imread("/content/dataset/test/11003.jpg", format='jpg').astype(np.uint8)
f = plt.figure(figsize=(8, 8))
plt.imshow(image)
plt.show()

In [ ]:
g = tf.Graph()
with g.as_default():
  new_model = tf.keras.models.load_model(os.path.join(export_dir, "saved_model"))
  new_model = new_model.signatures['serving_default']
  output_dict = new_model(tf.convert_to_tensor(np.expand_dims(image, 0)))
  init_op = tf.group([tf.global_variables_initializer(), tf.tables_initializer()])
g.finalize()

# Create session and initialize.
session = tf.Session(graph=g)
session.run(init_op)
output_dict = session.run(output_dict)

In [ ]:
# load labels (this is some sort of a protobuf file)
category_index = label_map_util.create_category_index_from_labelmap("dataset/labelmap.pbtxt", use_display_name=True)

In [ ]:
image_with_box = image.copy()
vis_util.visualize_boxes_and_labels_on_image_array(
  image_with_box,
  np.array(output_dict['detection_boxes'])[0],
  np.array(output_dict['detection_classes']).astype('int')[0],
  np.array(output_dict['detection_scores'])[0],
  category_index,
  use_normalized_coordinates=True,
  line_thickness=3)

f = plt.figure(figsize=(8, 8))
plt.imshow(image_with_box)
plt.show()

In [ ]:
# !zip my_model_inference_precomputed.zip -r /content/models/my_model_inference/

In [ ]:
boxes = output_dict['detection_boxes']
i=0
height, width = image.shape[:2]
ymin = int(boxes[0,i,0] * height)
xmin = int(boxes[0,i,1] * width)
ymax = int(boxes[0,i,2] * height)
xmax = int(boxes[0,i,3] * width)
cropped_image = tf.image.crop_to_bounding_box(image, ymin, xmin, 
                                       ymax - ymin, xmax - xmin)

sess = tf.Session()
with sess.as_default():
  cropped_image = tf.image.crop_to_bounding_box(image, ymin, xmin, 
                                       ymax - ymin, xmax - xmin).eval()

In [ ]:
Image.fromarray(cropped_image)